In [2]:
import pandas as pd
from pulp import LpProblem, LpVariable, LpBinary, LpMinimize, lpSum
from sklearn.preprocessing import MinMaxScaler
from geopy.distance import geodesic
import folium

In [3]:
bus_df = pd.read_csv("../노선_요약.csv")
subway_df = pd.read_csv("../지하철_요약.csv")
cafe_df = pd.read_csv("../일반음식점(카페)현황.csv", encoding="CP949")
park_df = pd.read_csv("../도시공원정보현황(제공표준).csv", encoding="CP949")
trash_bin_df = pd.read_csv("../경기도 성남시_쓰레기통_설치현황_20250325.csv")

In [4]:
cafe_df = cafe_df[(cafe_df['시군명'] == '성남시') & (cafe_df['영업상태명'] == '영업')]
cafe_df = cafe_df.rename(columns={'WGS84위도': '위도', 'WGS84경도': '경도', '사업장명': '이름'})

park_df = park_df[park_df['소재지지번주소'].str.startswith('경기도 성남시')].dropna(subset=['위도', '경도'])
park_df = park_df.rename(columns={'공원명': '이름'})

subway_df = subway_df.rename(columns={'역사명': '이름'})
bus_df = bus_df.dropna().rename(columns={'정류장명': '이름'})

In [5]:
scaler = MinMaxScaler()

cafe_df['가중치'] = scaler.fit_transform(cafe_df[['총시설규모(㎡)']])

park_df['가중치'] = scaler.fit_transform(park_df[['공원면적(㎡)']])

bus_df['혼잡도'] = bus_df['환승시간(분)'] * bus_df['노선개수']
bus_df['가중치'] = scaler.fit_transform(bus_df[['혼잡도']])

subway_df['일평균승하차인원'] = subway_df['승차총승객수'] + subway_df['하차총승객수']
subway_df['가중치'] = scaler.fit_transform(subway_df[['일평균승하차인원']])

In [6]:
demand_df = pd.concat([
    cafe_df[['위도', '경도', '이름', '가중치']],
    park_df[['위도', '경도', '이름', '가중치']],
    subway_df[['위도', '경도', '이름', '가중치']],
    bus_df[['위도', '경도', '이름', '가중치']]
], ignore_index=True)

demand_points = list(zip(demand_df['위도'], demand_df['경도'], demand_df['가중치'], demand_df['이름']))

In [7]:
existing_bins = set(zip(trash_bin_df['위도'].round(6), trash_bin_df['경도'].round(6)))

new_candidates_df = pd.concat([
    park_df[['위도', '경도', '이름']],
    bus_df[['위도', '경도', '이름']],
    cafe_df[['위도', '경도', '이름']],
    subway_df[['위도', '경도', '이름']]
], ignore_index=True)

new_candidates_df['좌표'] = list(zip(new_candidates_df['위도'].round(6), new_candidates_df['경도'].round(6)))
new_candidates_df = new_candidates_df[~new_candidates_df['좌표'].isin(existing_bins)]

candidate_bins = list(zip(new_candidates_df['위도'], new_candidates_df['경도'], new_candidates_df['이름']))

밑에 실행시간이 너무 오래걸려서 샘플링해서 결과 도출하는 코드

In [ ]:
# from pulp import LpProblem, LpVariable, LpBinary, LpMinimize, lpSum
# from geopy.distance import geodesic
# import random

# demand_sample = random.sample(demand_points, min(500, len(demand_points)))
# candidate_sample = random.sample(candidate_bins, min(100, len(candidate_bins)))

# distance_matrix = [
#     [geodesic(dp[:2], cb[:2]).meters for cb in candidate_sample]
#     for dp in demand_sample
# ]

# num_demands = len(demand_sample)
# num_candidates = len(candidate_sample)
# p = 10 

# x = [LpVariable(f"x_{j}", cat=LpBinary) for j in range(num_candidates)]
# assign = [[LpVariable(f"assign_{i}_{j}", cat=LpBinary) for j in range(num_candidates)] for i in range(num_demands)]
# z = LpVariable("z", lowBound=0, cat="Continuous")

# model = LpProblem("Fast_P_Center", LpMinimize)
# model += z

# for i in range(num_demands):
#     model += lpSum(assign[i][j] for j in range(num_candidates)) == 1
#     for j in range(num_candidates):
#         model += assign[i][j] <= x[j]
#     model += lpSum(distance_matrix[i][j] * assign[i][j] for j in range(num_candidates)) <= z

# model += lpSum(x) <= p

# model.solve()

# selected_locations = []
# for j in range(num_candidates):
#     if x[j].varValue > 0.5:
#         lat, lon, name = candidate_sample[j]
#         selected_locations.append((lat, lon, name))

# print(f"\n✅ 설치 위치 {len(selected_locations)}개 | 최대 커버 거리: {z.varValue:.2f}m")
# for loc in selected_locations[:5]:
#     print(f"설치 위치: {loc[2]} @ ({loc[0]:.5f}, {loc[1]:.5f})")


In [ ]:
from pulp import LpProblem, LpVariable, LpBinary, LpMinimize, lpSum
from geopy.distance import geodesic

distance_matrix = [
    [geodesic(dp[:2], cb[:2]).meters for cb in candidate_bins]
    for dp in demand_points
]

num_demands = len(demand_points)
num_candidates = len(candidate_bins)
p = 30  # 추가 설치 수

x = [LpVariable(f"x_{j}", cat=LpBinary) for j in range(num_candidates)]
assign = [[LpVariable(f"assign_{i}_{j}", cat=LpBinary) for j in range(num_candidates)] for i in range(num_demands)]
z = LpVariable("z", lowBound=0, cat="Continuous")

model = LpProblem("P_Center Model", LpMinimize)
model += z

In [ ]:
for i in range(num_demands):
    model += lpSum(assign[i][j] for j in range(num_candidates)) == 1
    for j in range(num_candidates):
        model += assign[i][j] <= x[j]
    model += lpSum(distance_matrix[i][j] * assign[i][j] for j in range(num_candidates)) <= z

model += lpSum(x) <= p

model.solve()

In [ ]:
selected_locations = []
for j in range(num_candidates):
    if x[j].varValue > 0.5:
        lat, lon, name = candidate_bins[j]
        selected_locations.append((lat, lon, name))

print(f"\n[신규 설치 : {len(selected_locations)}개 | 최대 커버 거리: {z.varValue:.2f}m]")
for loc in selected_locations[:10]:  # 상위 10개
    print(f"설치 위치: {loc[2]} @ ({loc[0]:.5f}, {loc[1]:.5f})")

In [10]:
import folium

center_lat = sum([pt[0] for pt in demand_points]) / len(demand_points)
center_lon = sum([pt[1] for pt in demand_points]) / len(demand_points)

m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

for lat, lon, weight, name in demand_points:
    folium.CircleMarker(
        location=[lat, lon],
        radius=4 + weight * 10,
        color='gray',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        tooltip=name
    ).add_to(m)

for lat, lon, name in selected_locations:
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color='red', icon='trash', prefix='fa'),
        tooltip=name
    ).add_to(m)

m 